## 📦 1. Imports et Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import pickle

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Imports réussis!")

## 📂 2. Création des Dossiers

In [ ]:
# Créer les dossiers si nécessaire
folders = [
    'results/figures',
    'results/metrics',
    'results/comparison'
]

for folder in folders:
    Path(folder).mkdir(parents=True, exist_ok=True)
    
print("✅ Dossiers créés!")

## 📥 3. Chargement des Résultats de Tous les Modèles

In [ ]:
print("📥 Chargement des résultats...\n")

# Liste des modèles
models = ['lstm', 'bilstm_attention', 'cnn_bilstm_attention', 'bert']
model_names = ['LSTM', 'BiLSTM+Attention', 'CNN-BiLSTM+Attention', 'BERT']

# Charger les résultats
results_dict = {}

for model, name in zip(models, model_names):
    try:
        with open(f'results/metrics/{model}_results.json', 'r') as f:
            results_dict[name] = json.load(f)
        print(f"✅ {name} chargé")
    except FileNotFoundError:
        print(f"⚠️ {name} non trouvé (results/metrics/{model}_results.json)")

print(f"\n📊 Modèles chargés: {len(results_dict)}")

## 📊 4. Tableau Comparatif Complet

In [ ]:
# Créer un DataFrame de comparaison
comparison_data = []

for model_name, results in results_dict.items():
    metrics = results['metrics']
    
    row = {
        'Modèle': model_name,
        'F1 (micro)': metrics['f1_micro'],
        'F1 (macro)': metrics['f1_macro'],
        'Precision (micro)': metrics['precision_micro'],
        'Precision (macro)': metrics['precision_macro'],
        'Recall (micro)': metrics['recall_micro'],
        'Recall (macro)': metrics['recall_macro'],
        'Hamming Loss': metrics['hamming_loss'],
        'Subset Accuracy': metrics['subset_accuracy'],
        'Params (M)': results['total_params'] / 1_000_000,
        'Temps (min)': results['training_time_minutes']
    }
    comparison_data.append(row)

# Créer le DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Trier par F1 micro
comparison_df = comparison_df.sort_values('F1 (micro)', ascending=False)

print("="*80)
print("📊 TABLEAU COMPARATIF DES MODÈLES")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Sauvegarder
comparison_df.to_csv('results/comparison/models_comparison.csv', index=False)
print("\n✅ Tableau sauvegardé!")

## 🏆 5. Meilleur Modèle et Statistiques

In [ ]:
# Identifier le meilleur modèle
best_model = comparison_df.iloc[0]

print("\n" + "="*60)
print("🏆 MEILLEUR MODÈLE")
print("="*60)
print(f"Modèle: {best_model['Modèle']}")
print(f"F1-Score (micro): {best_model['F1 (micro)']:.4f}")
print(f"F1-Score (macro): {best_model['F1 (macro)']:.4f}")
print(f"Precision (micro): {best_model['Precision (micro)']:.4f}")
print(f"Recall (micro): {best_model['Recall (micro)']:.4f}")
print(f"Hamming Loss: {best_model['Hamming Loss']:.4f}")
print(f"Paramètres: {best_model['Params (M)']:.2f}M")
print(f"Temps d'entraînement: {best_model['Temps (min)']:.2f} min")
print("="*60)

# Amélioration par rapport au baseline (LSTM)
if 'LSTM' in comparison_df['Modèle'].values:
    lstm_f1 = comparison_df[comparison_df['Modèle'] == 'LSTM']['F1 (micro)'].values[0]
    best_f1 = best_model['F1 (micro)']
    improvement = ((best_f1 - lstm_f1) / lstm_f1) * 100
    
    print(f"\n📈 Amélioration par rapport au baseline LSTM: +{improvement:.2f}%")

## 📊 6. Visualisations Comparatives

### 6.1 Graphique en Barres - Métriques Principales

In [ ]:
# Préparer les données pour le graphique
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = [
    ('F1 (micro)', 'F1-Score (Micro)'),
    ('F1 (macro)', 'F1-Score (Macro)'),
    ('Precision (micro)', 'Precision (Micro)'),
    ('Recall (micro)', 'Recall (Micro)')
]

for idx, (metric, title) in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    # Créer le barplot
    bars = ax.bar(comparison_df['Modèle'], comparison_df[metric], alpha=0.7, edgecolor='black')
    
    # Colorer le meilleur en vert
    max_idx = comparison_df[metric].idxmax()
    bars[max_idx].set_color('green')
    
    # Configuration
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12)
    ax.set_ylim([0, 1])
    ax.grid(axis='y', alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    # Ajouter les valeurs sur les barres
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('results/comparison/metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique des métriques sauvegardé!")

### 6.2 Radar Chart - Vue d'ensemble

In [ ]:
from math import pi

# Préparer les données pour le radar chart
categories = ['F1 (micro)', 'F1 (macro)', 'Precision (micro)', 'Recall (micro)']
num_vars = len(categories)

# Calculer les angles
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

# Créer le plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Couleurs pour chaque modèle
colors = ['blue', 'red', 'green', 'orange']

# Tracer chaque modèle
for idx, (_, row) in enumerate(comparison_df.iterrows()):
    values = [row[cat] for cat in categories]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=row['Modèle'], color=colors[idx])
    ax.fill(angles, values, alpha=0.15, color=colors[idx])

# Configuration
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12)
ax.set_ylim(0, 1)
ax.set_title('Radar Chart - Comparaison des Modèles', size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.savefig('results/comparison/radar_chart.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Radar chart sauvegardé!")

### 6.3 Trade-off Complexité vs Performance

In [ ]:
# Scatter plot: Paramètres vs F1-Score
fig, ax = plt.subplots(figsize=(12, 8))

# Plot les points
for idx, row in comparison_df.iterrows():
    ax.scatter(row['Params (M)'], row['F1 (micro)'], 
               s=500, alpha=0.6, edgecolors='black', linewidth=2)
    
    # Ajouter les labels
    ax.annotate(row['Modèle'], 
                (row['Params (M)'], row['F1 (micro)']),
                fontsize=12, fontweight='bold',
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3))

# Configuration
ax.set_xlabel('Nombre de Paramètres (Millions)', fontsize=14, fontweight='bold')
ax.set_ylabel('F1-Score (Micro)', fontsize=14, fontweight='bold')
ax.set_title('Trade-off: Complexité du Modèle vs Performance', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/comparison/complexity_vs_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique complexité vs performance sauvegardé!")

### 6.4 Temps d'entraînement vs Performance

In [ ]:
# Scatter plot: Temps vs F1-Score
fig, ax = plt.subplots(figsize=(12, 8))

# Plot les points
for idx, row in comparison_df.iterrows():
    ax.scatter(row['Temps (min)'], row['F1 (micro)'], 
               s=500, alpha=0.6, edgecolors='black', linewidth=2)
    
    # Ajouter les labels
    ax.annotate(row['Modèle'], 
                (row['Temps (min)'], row['F1 (micro)']),
                fontsize=12, fontweight='bold',
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.5))

# Configuration
ax.set_xlabel('Temps d\'entraînement (minutes)', fontsize=14, fontweight='bold')
ax.set_ylabel('F1-Score (Micro)', fontsize=14, fontweight='bold')
ax.set_title('Trade-off: Temps d\'entraînement vs Performance', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/comparison/time_vs_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique temps vs performance sauvegardé!")

## 📈 7. Analyse par Classe (Top/Bottom Émotions)

In [ ]:
print("📈 Analyse des performances par classe...\n")

# Charger les résultats par classe de chaque modèle
per_class_results = {}

for model, name in zip(models, model_names):
    try:
        df = pd.read_csv(f'results/metrics/{model}_per_class.csv')
        per_class_results[name] = df
        print(f"✅ {name} per-class chargé")
    except FileNotFoundError:
        print(f"⚠️ {name} per-class non trouvé")

if len(per_class_results) > 0:
    # Trouver les émotions communes
    first_model = list(per_class_results.values())[0]
    emotions = first_model['Emotion'].tolist()
    
    print(f"\n📊 {len(emotions)} émotions analysées")

### 7.1 Top 10 et Bottom 10 Émotions

In [ ]:
# Comparer les F1-scores pour chaque émotion
emotion_comparison = pd.DataFrame({'Emotion': emotions})

for model_name, df in per_class_results.items():
    emotion_comparison[model_name] = df.set_index('Emotion').loc[emotions, 'F1-Score'].values

# Calculer la moyenne
emotion_comparison['Moyenne'] = emotion_comparison[model_names].mean(axis=1)

# Top 10 émotions
top_10 = emotion_comparison.nlargest(10, 'Moyenne')

print("\n📊 TOP 10 Émotions (Meilleures performances moyennes):")
print(top_10.to_string(index=False))

# Bottom 10 émotions
bottom_10 = emotion_comparison.nsmallest(10, 'Moyenne')

print("\n📊 BOTTOM 10 Émotions (Pires performances moyennes):")
print(bottom_10.to_string(index=False))

# Sauvegarder
emotion_comparison.sort_values('Moyenne', ascending=False).to_csv(
    'results/comparison/emotion_comparison.csv', index=False
)
print("\n✅ Comparaison par émotion sauvegardée!")

### 7.2 Visualisation des Top/Bottom Émotions

In [ ]:
# Visualiser Top 10 et Bottom 10
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Top 10
top_10_sorted = top_10.sort_values('Moyenne')
x_pos = np.arange(len(top_10_sorted))
width = 0.2

for idx, model_name in enumerate(model_names):
    axes[0].barh(x_pos + idx * width, top_10_sorted[model_name], 
                 width, label=model_name, alpha=0.8)

axes[0].set_yticks(x_pos + width * 1.5)
axes[0].set_yticklabels(top_10_sorted['Emotion'])
axes[0].set_xlabel('F1-Score', fontsize=12)
axes[0].set_title('Top 10 Émotions - Comparaison des Modèles', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(axis='x', alpha=0.3)

# Bottom 10
bottom_10_sorted = bottom_10.sort_values('Moyenne')
x_pos = np.arange(len(bottom_10_sorted))

for idx, model_name in enumerate(model_names):
    axes[1].barh(x_pos + idx * width, bottom_10_sorted[model_name], 
                 width, label=model_name, alpha=0.8)

axes[1].set_yticks(x_pos + width * 1.5)
axes[1].set_yticklabels(bottom_10_sorted['Emotion'])
axes[1].set_xlabel('F1-Score', fontsize=12)
axes[1].set_title('Bottom 10 Émotions - Comparaison des Modèles', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower right')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('results/comparison/top_bottom_emotions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique top/bottom émotions sauvegardé!")

## 🔬 8. Étude d'Ablation

Analyser l'impact des différentes composantes architecturales.

In [ ]:
print("\n" + "="*60)
print("🔬 ÉTUDE D'ABLATION")
print("="*60)

# Comparer les modèles pour voir l'impact des composantes
if 'LSTM' in comparison_df['Modèle'].values:
    lstm_f1 = comparison_df[comparison_df['Modèle'] == 'LSTM']['F1 (micro)'].values[0]
    print(f"\nBaseline LSTM: {lstm_f1:.4f}")
    
    # Impact de l'attention (BiLSTM vs LSTM)
    if 'BiLSTM+Attention' in comparison_df['Modèle'].values:
        bilstm_f1 = comparison_df[comparison_df['Modèle'] == 'BiLSTM+Attention']['F1 (micro)'].values[0]
        attention_gain = bilstm_f1 - lstm_f1
        print(f"\n+ BiLSTM + Attention: {bilstm_f1:.4f} (gain: +{attention_gain:.4f})")
    
    # Impact du CNN (CNN-BiLSTM vs BiLSTM)
    if 'CNN-BiLSTM+Attention' in comparison_df['Modèle'].values and 'BiLSTM+Attention' in comparison_df['Modèle'].values:
        cnn_bilstm_f1 = comparison_df[comparison_df['Modèle'] == 'CNN-BiLSTM+Attention']['F1 (micro)'].values[0]
        cnn_gain = cnn_bilstm_f1 - bilstm_f1
        print(f"+ CNN layers: {cnn_bilstm_f1:.4f} (gain: +{cnn_gain:.4f})")
    
    # Impact de BERT
    if 'BERT' in comparison_df['Modèle'].values:
        bert_f1 = comparison_df[comparison_df['Modèle'] == 'BERT']['F1 (micro)'].values[0]
        bert_gain = bert_f1 - lstm_f1
        print(f"+ BERT (Transformers): {bert_f1:.4f} (gain: +{bert_gain:.4f})")

print("="*60)

# Visualiser l'ablation
ablation_data = []
if 'LSTM' in comparison_df['Modèle'].values:
    ablation_data.append(('LSTM\n(Baseline)', lstm_f1))
if 'BiLSTM+Attention' in comparison_df['Modèle'].values:
    ablation_data.append(('+ BiLSTM\n+ Attention', bilstm_f1))
if 'CNN-BiLSTM+Attention' in comparison_df['Modèle'].values:
    ablation_data.append(('+ CNN', cnn_bilstm_f1))
if 'BERT' in comparison_df['Modèle'].values:
    ablation_data.append(('BERT\n(Pre-trained)', bert_f1))

if len(ablation_data) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    labels, values = zip(*ablation_data)
    colors = ['lightblue', 'lightgreen', 'lightcoral', 'gold'][:len(labels)]
    
    bars = ax.bar(labels, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    
    # Ajouter les valeurs
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax.set_ylabel('F1-Score (Micro)', fontsize=14, fontweight='bold')
    ax.set_title('Étude d\'Ablation - Impact des Composantes', fontsize=16, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/comparison/ablation_study.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Graphique d'ablation sauvegardé!")

## 💡 9. Recommandations et Conclusions

In [ ]:
print("\n" + "="*70)
print("💡 RECOMMANDATIONS ET CONCLUSIONS")
print("="*70)

# Générer les recommandations basées sur les résultats
recommendations = []

# Meilleur modèle global
best_model_name = comparison_df.iloc[0]['Modèle']
recommendations.append(f"🏆 Meilleur modèle global: {best_model_name}")
recommendations.append(f"   F1-Score: {comparison_df.iloc[0]['F1 (micro)']:.4f}")

# Modèle le plus efficace (meilleur ratio performance/complexité)
comparison_df['Efficiency'] = comparison_df['F1 (micro)'] / (comparison_df['Params (M)'] + 1)
most_efficient = comparison_df.iloc[comparison_df['Efficiency'].idxmax()]
recommendations.append(f"\n⚡ Modèle le plus efficace: {most_efficient['Modèle']}")
recommendations.append(f"   Ratio performance/complexité: {most_efficient['Efficiency']:.4f}")

# Modèle le plus rapide
fastest_model = comparison_df.iloc[comparison_df['Temps (min)'].idxmin()]
recommendations.append(f"\n🚀 Modèle le plus rapide: {fastest_model['Modèle']}")
recommendations.append(f"   Temps: {fastest_model['Temps (min)']:.2f} minutes")

# Recommandations selon le cas d'usage
recommendations.append("\n📋 Recommandations selon le cas d'usage:")
recommendations.append("\n1. Production (haute performance requise):")
recommendations.append(f"   → {best_model_name}")
recommendations.append("   → Meilleur F1-Score, acceptable en termes de ressources")

recommendations.append("\n2. Environnement avec ressources limitées:")
recommendations.append(f"   → {most_efficient['Modèle']}")
recommendations.append("   → Bon compromis performance/complexité")

recommendations.append("\n3. Prototypage rapide / Développement:")
recommendations.append(f"   → {fastest_model['Modèle']}")
recommendations.append("   → Entraînement rapide pour itérations")

# Insights sur les émotions
if len(per_class_results) > 0:
    recommendations.append("\n📊 Insights sur les émotions:")
    recommendations.append(f"   - Top émotion: {top_10.iloc[0]['Emotion']} (F1: {top_10.iloc[0]['Moyenne']:.4f})")
    recommendations.append(f"   - Émotion difficile: {bottom_10.iloc[0]['Emotion']} (F1: {bottom_10.iloc[0]['Moyenne']:.4f})")
    recommendations.append("   → Concentrer les améliorations sur les émotions difficiles")

# Afficher les recommandations
for rec in recommendations:
    print(rec)

print("="*70)

# Sauvegarder les recommandations
with open('results/comparison/recommendations.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(recommendations))

print("\n✅ Recommandations sauvegardées!")

## 📄 10. Génération du Rapport Final

In [ ]:
# Générer un rapport markdown complet
report = f"""# 📊 Rapport Final - Détection d'Émotions Multi-Label

## 1. Vue d'ensemble du Projet

**Dataset**: GoEmotions (28 classes d'émotions)

**Modèles entraînés**: {len(comparison_df)}
- {', '.join(comparison_df['Modèle'].tolist())}

---

## 2. Résultats Comparatifs

### 2.1 Tableau des Performances

{comparison_df.to_markdown(index=False)}

### 2.2 Meilleur Modèle

**🏆 {best_model_name}**
- F1-Score (micro): {best_model['F1 (micro)']:.4f}
- F1-Score (macro): {best_model['F1 (macro)']:.4f}
- Precision (micro): {best_model['Precision (micro)']:.4f}
- Recall (micro): {best_model['Recall (micro)']:.4f}
- Paramètres: {best_model['Params (M)']:.2f}M
- Temps d'entraînement: {best_model['Temps (min)']:.2f} min

---

## 3. Analyse des Émotions

### 3.1 Top 10 Émotions (meilleures performances)

{top_10[['Emotion', 'Moyenne']].to_markdown(index=False)}

### 3.2 Bottom 10 Émotions (performances à améliorer)

{bottom_10[['Emotion', 'Moyenne']].to_markdown(index=False)}

---

## 4. Recommandations

{chr(10).join(recommendations)}

---

## 5. Fichiers Générés

- Résultats des modèles: `results/metrics/`
- Graphiques: `results/figures/`
- Comparaisons: `results/comparison/`

---

**Date**: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Sauvegarder le rapport
with open('results/comparison/final_report.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("✅ Rapport final généré: results/comparison/final_report.md")

## 🎉 11. Résumé Final

In [ ]:
print("\n" + "="*70)
print("🎉 ANALYSE COMPARATIVE TERMINÉE!")
print("="*70)
print(f"\n✅ {len(comparison_df)} modèles comparés")
print(f"✅ {len(emotions) if len(per_class_results) > 0 else 'N/A'} émotions analysées")
print(f"✅ {len([f for f in Path('results/comparison').glob('*.png')])} graphiques générés")
print(f"\n📂 Tous les résultats sont dans:")
print(f"   - results/comparison/")
print(f"   - results/figures/")
print(f"   - results/metrics/")

print("\n📊 Fichiers clés:")
print("   - models_comparison.csv: Tableau comparatif")
print("   - final_report.md: Rapport complet")
print("   - recommendations.txt: Recommandations")
print("   - *.png: Tous les graphiques")

print("\n🏆 Meilleur modèle: " + best_model_name)
print(f"   F1-Score: {best_model['F1 (micro)']:.4f}")

print("\n" + "="*70)
print("🎯 Projet de Détection d'Émotions - COMPLET!")
print("="*70)

## 🔍 11. Analyse d'Explicabilité (Partie 5)

Utilisation de LIME pour expliquer les prédictions du modèle sur des exemples concrets.


In [ ]:
# Installation de LIME
!pip install -q lime

print("✅ LIME installé!")


In [ ]:
from lime.lime_text import LimeTextExplainer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

print("🔮 Démarrage de l'analyse d'explicabilité (LIME)...")

# 1. Charger les ressources nécessaires
DATA_PATH = 'data/processed'
MODEL_PATH = 'models/lstm/best_model.h5'

try:
    # Charger le tokenizer
    with open(f'{DATA_PATH}/tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    
    # Charger les métadonnées pour labels et max_len
    with open(f'{DATA_PATH}/metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
        classes = metadata['emotion_labels']
        max_len = metadata['max_sequence_length']

    # Charger le modèle LSTM (plus rapide pour l'inférence LIME)
    model = load_model(MODEL_PATH)
    print("✅ Modèle LSTM et ressources chargés")
    
    # Fonction de prédiction pour LIME
    def predict_proba(texts):
        # Tokenization et padding
        seqs = tokenizer.texts_to_sequences(texts)
        padded = pad_sequences(seqs, maxlen=max_len, padding='post', truncating='post')
        # Prédiction
        return model.predict(padded)

    # Initialiser l'explainer
    explainer = LimeTextExplainer(class_names=classes)
    
    # Textes exemples à expliquer
    test_texts = [
        "I am so happy and excited about this new project! It feels amazing.", # Joy/Excitement
        "This is absolutely terrible, I hate how they treated you.", # Anger/Disgust
        "I'm really worried about the exam tomorrow, I feel sick.", # Fear/Nervousness
        "Thank you so much for your help, I really appreciate it." # Gratitude
    ]
    
    print(f"\n🧠 Analyse de {len(test_texts)} exemples...")

    for idx, text in enumerate(test_texts):
        print(f"\n📝 Exemple {idx+1}: '{text}'")
        
        # Générer l'explication (top 2 labels)
        exp = explainer.explain_instance(text, predict_proba, num_features=6, top_labels=2)
        
        # Afficher les top classes prédites
        probs = predict_proba([text])[0]
        top_indices = probs.argsort()[-3:][::-1]
        print("  Top prédictions:")
        for i in top_indices:
            print(f"  - {classes[i]}: {probs[i]:.4f}")
            
        # Sauvegarder la visualisation HTML
        exp.save_to_file(f'results/figures/lime_explanation_{idx+1}.html')
        print(f"  ✅ Explication sauvegardée: results/figures/lime_explanation_{idx+1}.html")
        
        # Afficher les features importantes (liste)
        print("  Mots impactants:")
        for label_idx in exp.available_labels():
            label_name = classes[label_idx]
            print(f"    Pour '{label_name}': {exp.as_list(label=label_idx)}")

except Exception as e:
    print(f"⚠️ Erreur lors de l'analyse LIME: {e}")
    print("Assurez-vous que le modèle LSTM est bien entraîné et sauvegardé.")
